In [1]:
import pandas as pd
import numpy as np

import sklearn as sk
from sklearn import preprocessing
from MulticoreTSNE import MulticoreTSNE as TSNE

import holoviews as hv
from bokeh.models import HoverTool

hv.extension('bokeh')

In [2]:
datasets = pd.read_csv("../pilotage/data/datasets.csv", sep=";")

/home/tk/.virtualenvs/notebooks/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
list(datasets)

['Unnamed: 0',
 '_cls',
 '_id',
 'acronym',
 'badges',
 'created_at',
 'deleted',
 'description',
 'ext',
 'extras',
 'featured',
 'frequency',
 'frequency_date',
 'last_modified',
 'license',
 'metrics',
 'organization',
 'owner',
 'private',
 'resources',
 'slug',
 'spatial',
 'tags',
 'temporal_coverage',
 'title']

In [4]:
datasets.head()

,Unnamed: 0,_cls,_id,acronym,badges,created_at,deleted,description,ext,extras,...,metrics,organization,owner,private,resources,slug,spatial,tags,temporal_coverage,title
0,0,Dataset,53dfc8b4a3a729110ca8d360,NaN,NaN,2014-07-29 10:43:50.053,NaN,Aides perçues au titre de la PAC de 2010 à 2012,{},{'datagouv_ckan_id': '63bcc97e-18db-4c64-8935-...,...,"{'cow_score': 0, 'cow_warnings': 0, 'followers...",NaN,53d18de7a3a72946ca86f1ca,NaN,[],aides-percues-au-titre-de-la-politique-agricol...,"{'granularity': 'fr:commune', 'zones': []}","['2010', '2011', '2012', 'pac']","{'start': '2010-01-01', 'end': '2012-12-31'}",Aides perçues au titre de la Politique Agricol...
1,1,Dataset,537d599ca3a72973a2dc0276,NaN,NaN,2014-05-21 07:14:16.618,NaN,Le diagnostic des cales met en évidence les co...,{},{'datagouv_ckan_id': 'a2358c63-313d-4917-af2f-...,...,"{'cow_criticals': 0, 'cow_errors': 1, 'cow_sco...",NaN,537d58d0a3a72973a2dc0267,NaN,[],acces-a-la-mer-de-lorient-agglomeration,"{'granularity': 'fr:epci', 'zones': []}","['cales', 'donnees-publiques', 'littoral', 'lo...",NaN,Accès à la mer de Lorient Agglomération
2,2,Dataset,53698e89a3a729239d2034c0,NaN,NaN,2013-10-20 14:15:12.734,NaN,"Liste des événements (expositions, spectacles,...",{},{'Date de production des données': '17/09/2013...,...,"{'cow_criticals': 0, 'cow_errors': 0, 'cow_sco...",534fff74a3a7292c64a77ddd,NaN,False,"[{'created_at': datetime.datetime(2014, 5, 7, ...",agenda-des-evenements-du-chateau-de-versailles...,"{'zones': ['fr:commune:78646@1942-01-01'], 'gr...",['culture'],NaN,Agenda des événements du Château de Versailles
3,3,Dataset,53699760a3a729239d204c97,NaN,NaN,2014-04-18 08:00:26.168,NaN,Jeu TCL,{},{'datagouv_ckan_id': '308b56f3-89ec-4b7a-9f03-...,...,"{'cow_criticals': 0, 'cow_errors': 0, 'cow_sco...",NaN,53542173a3a7294fcb493bc1,NaN,"[{'created_at': datetime.datetime(2014, 5, 7, ...",jeu-tcl,NaN,[],NaN,Jeu TCL
4,4,Dataset,53699042a3a729239d20396d,NaN,NaN,2014-02-26 15:29:48.643,NaN,"Les immeubles de l’État sont nombreux, variés,...",{},{'datagouv_ckan_id': '0e6cd4f9-e8b9-4c28-8261-...,...,"{'cow_criticals': 0, 'cow_errors': 0, 'cow_sco...",534fff8ea3a7292c64a77f02,NaN,NaN,"[{'created_at': datetime.datetime(2014, 5, 7, ...",catalogue-des-sites-publics-ouverts-aux-evenem...,NaN,"['lieux-publics', 'evenements']",NaN,Catalogue des sites publics ouverts aux evenem...


In [5]:
def to_list(r):
    if pd.notnull(r["tags"]):
        return eval(r["tags"])
    else:
        return []

tags = datasets.apply(to_list, axis=1)

In [6]:
tags

0                                  [2010, 2011, 2012, pac]
1        [cales, donnees-publiques, littoral, lorient-a...
2                                                [culture]
3                                                       []
4                              [lieux-publics, evenements]
5                               [liste-de-jeux-de-donnees]
6                                [argiles, communes, sols]
7                                                       []
8                                                       []
9                                                       []
10       [ameublement, commerces, electromenager, decor...
11       [culture, loisirs, tourisme, patrimoine-culturel]
12                            [elections, departementales]
13       [parking, place, stationnement, voiture, trans...
14        [distributeur, automate, boisson, openstreetmap]
15       [arbres, arbres-remarquables, vegetation, espa...
16       [ville-sans-limite, urbanisme, concertation, p.

In [9]:
mlb = preprocessing.MultiLabelBinarizer()
m = mlb.fit_transform(tags.tolist())
mlb.classes_

array(['', '00-national-dgfip', '02-region-bourgogne', ..., 'zsp', 'zsup',
       'zus'], dtype=object)

In [11]:
XY = TSNE(n_components=2, verbose=100, n_jobs=2).fit_transform(m)

In [12]:
XY

array([[  1.02457562,   4.25201788],
       [  4.97712951, -12.94580485],
       [ -5.34095652, -30.16159953],
       ...,
       [-12.73980814,  24.55379228],
       [-10.58579406,  27.73464756],
       [-35.23691942,   6.61659904]])

In [28]:
## replace me wit joblib

import pickle

In [18]:
pickle.dump(XY, open("tsne.pck", "wb"))

In [46]:
datasets['x'] = XY[:,0]
datasets['y'] = XY[:,1]

In [53]:
scatter = hv.Scatter(datasets, kdims=["x", "y"], vdims=["title"])

tooltips = [
    ("title", "@title")
]

hover = HoverTool(tooltips=tooltips)

scatter.opts(tools=[hover], height=800, width=800, padding=0.1, xaxis=None, yaxis=None)

:Scatter   [x,y]   (title)